# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

#### Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.37.0


### Compute cluster

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'palsy'
project_folder = './palsy-project'


experiment=Experiment(ws, experiment_name)
print(experiment)

quick-starts-ws-178503
aml-quickstarts-178503
southcentralus
cdbe0b43-92a0-4715-838a-f2648cc7ad21
Experiment(Name: palsy,
Workspace: quick-starts-ws-178503)


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpucluster4#"#auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True) #min_node_count = 1, , timeout_in_minutes = 10
# For a more detailed view of current AmlCompute status, use get_status().

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

............

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# ws = Workspace.from_config()
# print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# # choose a name for experiment
# experiment_name = 'palsy'
# project_folder = './palsy-project'


# experiment=Experiment(ws, experiment_name)
# print(experiment)

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bellpalsy"
description_text = "Datapoints of face"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
#         datastore = Datastore.get(ws, datastore_name='palda')
#         example_data=datastore
        path="https://raw.githubusercontent.com/nsourlos/palda_azure/main/palda.csv"
#         example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(path=path)#path = [(datastore, 'palda.csv')])#example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df=dataset.to_pandas_dataframe()
# print(df)
df = df.iloc[: , :-1]

# df = dataset.to_pandas_dataframe()
df.describe()
print(df)

     x_1  y_1  x_2  y_2  x_3  y_3  x_4  y_4  x_5  y_5  ...  y_64  x_65  y_65  \
0    132  439  115  496  111  566  134  633  177  688  ...   581   588   583   
1    194  402  186  444  182  486  199  543  230  591  ...   490   542   507   
2     76  379   78  455   86  559  121  631  161  704  ...   628   654   629   
3    201  366  203  369  217  415  230  444  249  467  ...   418   487   419   
4    153  519  157  563  144  614  151  684  166  757  ...   672   520   686   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   ...   
198  227  226  234  257  244  287  254  315  268  340  ...   303   402   303   
199  131  168  128  184  125  200  123  217  125  233  ...   219   214   226   
200  132  231  134  254  136  278  139  303  148  327  ...   312   269   312   
201   62  103   63  116   64  127   66  139   69  151  ...   138   126   141   
202   -2  378    5  465   20  556   37  650   63  745  ...   660   531   673   

     x_66  y_66  x_67  y_67  x_68  y_68

In [5]:
# split df!!!!
from sklearn.model_selection import train_test_split
x_train,x_test=train_test_split(df,shuffle=True)#,df.iloc[:,-1],#x,y,test_size=0.2,random_state=42)
#,y_train,y_test
# print(x_train)
# print(y_train)
x_train.to_csv('x_train.csv',index=False)

datastore=ws.get_default_datastore()
datastore.upload_files(files=['./x_train.csv'])

train=Dataset.Tabular.from_delimited_files([(datastore,'x_train.csv')])

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Target already exists. Skipping upload for x_train.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

You can define outputs for the AutoMLStep using TrainingOutput

In [6]:
# from azureml.pipeline.core import PipelineData, TrainingOutput

# ds = ws.get_default_datastore()
# metrics_output_name = 'metrics_output'
# best_model_output_name = 'best_model_output'

# metrics_data = PipelineData(name='metrics_data',
#                            datastore=ds,
#                            pipeline_output_name=metrics_output_name,
#                            training_output=TrainingOutput(type='Metrics'))
# model_data = PipelineData(name='model_data',
#                            datastore=ds,
#                            pipeline_output_name=best_model_output_name,
#                            training_output=TrainingOutput(type='Model'))

In [7]:
# automl_step = AutoMLStep(
#     name='automl_module',
#     automl_config=automl_config,
#     outputs=[metrics_data, model_data],
#     allow_reuse=True)

In [8]:
# from azureml.pipeline.core import Pipeline
# pipeline = Pipeline(
#     description="pipeline_with_automlstep",
#     workspace=ws,    
#     steps=[automl_step])

In [9]:
# TODO: Put your automl settings here
#taken from https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted',
    "enable_onnx_compatible_models": True
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=train,#df
                             label_column_name="diagnosis",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)#,show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
palsy,AutoML_161e8d7b-0ced-4643-b13c-d20a022de3db,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [12]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
palsy,AutoML_161e8d7b-0ced-4643-b13c-d20a022de3db,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+------------------------------+
|Number of folds               |
+==============================+
|10                            |
+------------------------------+

********************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbal

{'runId': 'AutoML_161e8d7b-0ced-4643-b13c-d20a022de3db',
 'target': 'cpucluster4',
 'status': 'Completed',
 'startTimeUtc': '2022-01-25T14:14:30.75411Z',
 'endTimeUtc': '2022-01-25T14:33:11.26297Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpucluster4',
  'AMLSettingsJsonString': '{"path":null,"name":"palsy","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-178503","workspace_name":"quick-starts-ws-178503","region":"southcentralus","compute_target":"cpucluster4","spark_s

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [31]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
remote_run.get_metrics()

import onnxmltools
best_run, fitted_model = remote_run.get_output(return_onnx_model=True)
print(best_run.get_metrics())
print(best_run)
print(fitted_model)
# Taken from https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models

onnxmltools.utils.save_model(fitted_model, './best_automl.onnx')

{'accuracy': 0.8475000000000001, 'norm_macro_recall': 0.6282738095238094, 'precision_score_weighted': 0.8006212121212121, 'AUC_macro': 0.9331466558133223, 'balanced_accuracy': 0.7521825396825397, 'average_precision_score_micro': 0.9265458493936292, 'precision_score_micro': 0.8475000000000001, 'recall_score_macro': 0.7521825396825397, 'weighted_accuracy': 0.9042746544311688, 'AUC_weighted': 0.9384439356939357, 'recall_score_weighted': 0.8475000000000001, 'log_loss': 0.39703535429064296, 'average_precision_score_macro': 0.8733472909365766, 'AUC_micro': 0.9578715277777776, 'recall_score_micro': 0.8475000000000001, 'f1_score_macro': 0.7362077840173816, 'matthews_correlation': 0.7599297274029568, 'f1_score_weighted': 0.8130769800410862, 'f1_score_micro': 0.8475000000000001, 'average_precision_score_weighted': 0.9102310906685906, 'precision_score_macro': 0.7435016835016834, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_161e8d7b-0ced-4643-b13c-d20a022de3db_35/accuracy_table', 

In [14]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_name = "palda_dataset",
                        model_path = "./best_automl.onnx",
                        description = "Best AutoML Model",)

Registering model palda_dataset


In [16]:
model=remote_run.register_model(model_name = "palda_dataset",
                        description = "Best AutoML Model",)
                        #taken from jwood803 github

In [17]:
#taken from https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py
#and from https://stackoverflow.com/questions/66437607/how-to-create-azure-ml-inference-config-and-deployment-config-class-objects-from

from azureml.core.environment import Environment
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml") #environment instance creation
#we need these otherwise we will get error on non-known environment to run

script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py') #this exists before we run the commands in the cells above




from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file_name, environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               description = 'palsy classification service')

aci_service_name = 'palsy-classification2'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(show_output=True)



# InferenceConfig(entry_script=)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-25 14:36:08+00:00 Creating Container Registry if not exists.
2022-01-25 14:36:09+00:00 Registering the environment.
2022-01-25 14:36:10+00:00 Use the existing image.
2022-01-25 14:36:11+00:00 Generating deployment configuration.
2022-01-25 14:36:12+00:00 Submitting deployment to compute.
2022-01-25 14:36:14+00:00 Checking the status of deployment palsy-classification2..
2022-01-25 14:41:43+00:00 Checking the status of inference endpoint palsy-classification2.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [18]:
# published_pipeline = remote_run.publish_pipeline(
#     name="palda", description="palsy dataset", version="1.0")

# published_pipeline

In [19]:
# from azureml.core.authentication import InteractiveLoginAuthentication

# interactive_auth = InteractiveLoginAuthentication()
# auth_header = interactive_auth.get_authentication_header()



In [20]:
# import requests

# rest_endpoint = published_pipeline.endpoint
# response = requests.post(rest_endpoint, 
#                          headers=auth_header, 
#                          json={"ExperimentName": "pipeline-rest-endpoint"}
#                         )

In [21]:
# try:
#     response.raise_for_status()
# except Exception:    
#     raise Exception("Received bad response from the endpoint: {}\n"
#                     "Response Code: {}\n"
#                     "Headers: {}\n"
#                     "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

# run_id = response.json().get('Id')
# print('Submitted pipeline run: ', run_id)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [22]:
# print("Key: " + aci_service.get_keys()[0]) #[0]


In [23]:
x_test.iloc[16].to_dict() #0,2,16 - helathy peripheral central

{'x_1': 309,
 'y_1': 326,
 'x_2': 312,
 'y_2': 390,
 'x_3': 318,
 'y_3': 463,
 'x_4': 328,
 'y_4': 524,
 'x_5': 341,
 'y_5': 575,
 'x_6': 365,
 'y_6': 616,
 'x_7': 395,
 'y_7': 653,
 'x_8': 427,
 'y_8': 680,
 'x_9': 452,
 'y_9': 682,
 'x_10': 491,
 'y_10': 672,
 'x_11': 528,
 'y_11': 648,
 'x_12': 566,
 'y_12': 608,
 'x_13': 586,
 'y_13': 551,
 'x_14': 595,
 'y_14': 489,
 'x_15': 604,
 'y_15': 421,
 'x_16': 602,
 'y_16': 360,
 'x_17': 606,
 'y_17': 318,
 'x_18': 332,
 'y_18': 240,
 'x_19': 348,
 'y_19': 212,
 'x_20': 371,
 'y_20': 196,
 'x_21': 401,
 'y_21': 205,
 'x_22': 422,
 'y_22': 223,
 'x_23': 493,
 'y_23': 270,
 'x_24': 510,
 'y_24': 247,
 'x_25': 541,
 'y_25': 247,
 'x_26': 569,
 'y_26': 254,
 'x_27': 582,
 'y_27': 268,
 'x_28': 462,
 'y_28': 291,
 'x_29': 462,
 'y_29': 330,
 'x_30': 462,
 'y_30': 366,
 'x_31': 458,
 'y_31': 396,
 'x_32': 409,
 'y_32': 419,
 'x_33': 433,
 'y_33': 424,
 'x_34': 460,
 'y_34': 432,
 'x_35': 486,
 'y_35': 431,
 'x_36': 509,
 'y_36': 428,
 'x_37': 3

In [24]:
# aci_service.get_keys()

In [25]:
#endpoint.py file
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
# scoring_uri = 'http://7bbc8877-700c-4035-8097-c35c896039a2.southcentralus.azurecontainer.io/score'
scoring_uri=aci_service.scoring_uri
# If the service is authenticated, set the key or token
# key = 'Y9bgMCp5O0geM74pEThdQVLiaRMHBjrB'
# key=aci_service.get_keys()[0]

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {'x_1': 38,
            'y_1': 88,
            'x_2': 42,
            'y_2': 125,
            'x_3': 50,
            'y_3': 161,
            'x_4': 57,
            'y_4': 195,
            'x_5': 69,
            'y_5': 228,
            'x_6': 89,
            'y_6': 259,
            'x_7': 113,
            'y_7': 285,
            'x_8': 144,
            'y_8': 306,
            'x_9': 177,
            'y_9': 311,
            'x_10': 209,
            'y_10': 303,
            'x_11': 238,
            'y_11': 281,
            'x_12': 262,
            'y_12': 253,
            'x_13': 281,
            'y_13': 222,
            'x_14': 292,
            'y_14': 188,
            'x_15': 297,
            'y_15': 152,
            'x_16': 301,
            'y_16': 117,
            'x_17': 301,
            'y_17': 81,
            'x_18': 55,
            'y_18': 60,
            'x_19': 69,
            'y_19': 42,
            'x_20': 93,
            'y_20': 32,
            'x_21': 117,
            'y_21': 35,
            'x_22': 139,
            'y_22': 46,
            'x_23': 197,
            'y_23': 43,
            'x_24': 218,
            'y_24': 31,
            'x_25': 241,
            'y_25': 26,
            'x_26': 266,
            'y_26': 30,
            'x_27': 283,
            'y_27': 46,
            'x_28': 169,
            'y_28': 78,
            'x_29': 171,
            'y_29': 100,
            'x_30': 172,
            'y_30': 123,
            'x_31': 173,
            'y_31': 146,
            'x_32': 147,
            'y_32': 164,
            'x_33': 160,
            'y_33': 168,
            'x_34': 173,
            'y_34': 172,
            'x_35': 187,
            'y_35': 167,
            'x_36': 199,
            'y_36': 163,
            'x_37': 81,
            'y_37': 89,
            'x_38': 96,
            'y_38': 81,
            'x_39': 114,
            'y_39': 82,
            'x_40': 131,
            'y_40': 92,
            'x_41': 113,
            'y_41': 93,
            'x_42': 96,
            'y_42': 93,
            'x_43': 207,
            'y_43': 89,
            'x_44': 224,
            'y_44': 76,
            'x_45': 241,
            'y_45': 73,
            'x_46': 258,
            'y_46': 81,
            'x_47': 244,
            'y_47': 86,
            'x_48': 226,
            'y_48': 88,
            'x_49': 105,
            'y_49': 201,
            'x_50': 128,
            'y_50': 190,
            'x_51': 154,
            'y_51': 186,
            'x_52': 173,
            'y_52': 188,
            'x_53': 192,
            'y_53': 185,
            'x_54': 216,
            'y_54': 187,
            'x_55': 239,
            'y_55': 197,
            'x_56': 220,
            'y_56': 237,
            'x_57': 195,
            'y_57': 254,
            'x_58': 173,
            'y_58': 257,
            'x_59': 152,
            'y_59': 256,
            'x_60': 126,
            'y_60': 240,
            'x_61': 114,
            'y_61': 204,
            'x_62': 154,
            'y_62': 197,
            'x_63': 173,
            'y_63': 198,
            'x_64': 192,
            'y_64': 196,
            'x_65': 231,
            'y_65': 200,
            'x_66': 193,
            'y_66': 235,
            'x_67': 173,
            'y_67': 238,
            'x_68': 153,
            'y_68': 237     
          },
          {
            'x_1': 209,
            'y_1': 238,
            'x_2': 208,
            'y_2': 300,
            'x_3': 208,
            'y_3': 354,
            'x_4': 211,
            'y_4': 403,
            'x_5': 218,
            'y_5': 463,
            'x_6': 236,
            'y_6': 558,
            'x_7': 286,
            'y_7': 623,
            'x_8': 348,
            'y_8': 667,
            'x_9': 429,
            'y_9': 689,
            'x_10': 502,
            'y_10': 691,
            'x_11': 574,
            'y_11': 662,
            'x_12': 626,
            'y_12': 599,
            'x_13': 672,
            'y_13': 556,
            'x_14': 696,
            'y_14': 513,
            'x_15': 696,
            'y_15': 463,
            'x_16': 709,
            'y_16': 405,
            'x_17': 728,
            'y_17': 311,
            'x_18': 258,
            'y_18': 138,
            'x_19': 300,
            'y_19': 101,
            'x_20': 342,
            'y_20': 90,
            'x_21': 387,
            'y_21': 90,
            'x_22': 424,
            'y_22': 112,
            'x_23': 521,
            'y_23': 114,
            'x_24': 569,
            'y_24': 93,
            'x_25': 614,
            'y_25': 91,
            'x_26': 668,
            'y_26': 104,
            'x_27': 702,
            'y_27': 144,
            'x_28': 476,
            'y_28': 159,
            'x_29': 476,
            'y_29': 215,
            'x_30': 473,
            'y_30': 269,
            'x_31': 459,
            'y_31': 342,
            'x_32': 389,
            'y_32': 373,
            'x_33': 415,
            'y_33': 386,
            'x_34': 450,
            'y_34': 392,
            'x_35': 493,
            'y_35': 392,
            'x_36': 527,
            'y_36': 390,
            'x_37': 291,
            'y_37': 197,
            'x_38': 324,
            'y_38': 176,
            'x_39': 379,
            'y_39': 171,
            'x_40': 411,
            'y_40': 196,
            'x_41': 364,
            'y_41': 211,
            'x_42': 321,
            'y_42': 210,
            'x_43': 543,
            'y_43': 212,
            'x_44': 577,
            'y_44': 181,
            'x_45': 635,
            'y_45': 193,
            'x_46': 661,
            'y_46': 221,
            'x_47': 616,
            'y_47': 233,
            'x_48': 568,
            'y_48': 228,
            'x_49': 341,
            'y_49': 471,
            'x_50': 364,
            'y_50': 451,
            'x_51': 403,
            'y_51': 445,
            'x_52': 440,
            'y_52': 453,
            'x_53': 471,
            'y_53': 469,
            'x_54': 506,
            'y_54': 489,
            'x_55': 535,
            'y_55': 523,
            'x_56': 506,
            'y_56': 544,
            'x_57': 467,
            'y_57': 541,
            'x_58': 428,
            'y_58': 529,
            'x_59': 400,
            'y_59': 516,
            'x_60': 366,
            'y_60': 498,
            'x_61': 356,
            'y_61': 473,
            'x_62': 395,
            'y_62': 475,
            'x_63': 429,
            'y_63': 486,
            'x_64': 470,
            'y_64': 499,
            'x_65': 511,
            'y_65': 513,
            'x_66': 469,
            'y_66': 508,
            'x_67': 430,
            'y_67': 501,
            'x_68': 377,
            'y_68': 482,   
          },
           
           {'x_1': 124,
            'y_1': 348,
            'x_2': 134,
            'y_2': 415,
            'x_3': 154,
            'y_3': 494,
            'x_4': 179,
            'y_4': 568,
            'x_5': 219,
            'y_5': 617,
            'x_6': 263,
            'y_6': 663,
            'x_7': 323,
            'y_7': 696,
            'x_8': 385,
            'y_8': 711,
            'x_9': 466,
            'y_9': 713,
            'x_10': 550,
            'y_10': 710,
            'x_11': 609,
            'y_11': 681,
            'x_12': 674,
            'y_12': 631,
            'x_13': 724,
            'y_13': 584,
            'x_14': 756,
            'y_14': 538,
            'x_15': 778,
            'y_15': 489,
            'x_16': 789,
            'y_16': 426,
            'x_17': 786,
            'y_17': 356,
            'x_18': 172,
            'y_18': 257,
            'x_19': 221,
            'y_19': 224,
            'x_20': 284,
            'y_20': 220,
            'x_21': 341,
            'y_21': 230,
            'x_22': 392,
            'y_22': 265,
            'x_23': 533,
            'y_23': 246,
            'x_24': 593,
            'y_24': 217,
            'x_25': 634,
            'y_25': 218,
            'x_26': 685,
            'y_26': 224,
            'x_27': 729,
            'y_27': 245,
            'x_28': 453,
            'y_28': 283,
            'x_29': 455,
            'y_29': 336,
            'x_30': 459,
            'y_30': 387,
            'x_31': 463,
            'y_31': 454,
            'x_32': 367,
            'y_32': 457,
            'x_33': 413,
            'y_33': 466,
            'x_34': 470,
            'y_34': 485,
            'x_35': 522,
            'y_35': 466,
            'x_36': 568,
            'y_36': 444,
            'x_37': 228,
            'y_37': 313,
            'x_38': 271,
            'y_38': 291,
            'x_39': 323,
            'y_39': 288,
            'x_40': 361,
            'y_40': 308,
            'x_41': 315,
            'y_41': 327,
            'x_42': 265,
            'y_42': 322,
            'x_43': 550,
            'y_43': 304,
            'x_44': 605,
            'y_44': 277,
            'x_45': 663,
            'y_45': 280,
            'x_46': 703,
            'y_46': 297,
            'x_47': 655,
            'y_47': 312,
            'x_48': 594,
            'y_48': 312,
            'x_49': 326,
            'y_49': 547,
            'x_50': 377,
            'y_50': 529,
            'x_51': 425,
            'y_51': 525,
            'x_52': 478,
            'y_52': 524,
            'x_53': 535,
            'y_53': 517,
            'x_54': 585,
            'y_54': 512,
            'x_55': 644,
            'y_55': 518,
            'x_56': 609,
            'y_56': 551,
            'x_57': 557,
            'y_57': 579,
            'x_58': 483,
            'y_58': 594,
            'x_59': 424,
            'y_59': 596,
            'x_60': 377,
            'y_60': 588,
            'x_61': 342,
            'y_61': 551,
            'x_62': 427,
            'y_62': 542,
            'x_63': 484,
            'y_63': 540,
            'x_64': 558,
            'y_64': 527,
            'x_65': 624,
            'y_65': 527,
            'x_66': 545,
            'y_66': 566,
            'x_67': 478,
            'y_67': 577,
            'x_68': 400,
            'y_68': 578,  
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
#print(resp)
print(resp.json())


{"result": ["Healthy", "peripheral", "central"]}


TODO: In the cell below, print the logs of the web service and delete the service

In [26]:
aci_service.get_logs()

'2022-01-25T14:41:09,626206800+00:00 - rsyslog/run \n2022-01-25T14:41:09,635537100+00:00 - iot-server/run \n2022-01-25T14:41:09,641743000+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\n2022-01-25T14:41:09,652004200+00:00 - nginx/run \nrsyslogd: /azureml-envs/azureml_0f7f18983f8fadd52db7ab2a1965cfbb/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2022-01-25T14:41:10,260522100+00:00 - iot-server/finish 1 0\n2022-01-25T14:41:10,265130400+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (74)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 103\nSPARK_HOME not set. Skipping PySpark Initialization.\nGenerating new fontManager, this may take some time...\nInitializing logger\n2022-01-25 14:41:12,999 | root | INFO | Starting up app insights client

In [27]:
#logs.py file
# from azureml.core.webservice import Webservice

# # Set with the deployment name
# name = "palsy-classification2"

# # load existing web service
# service = Webservice(name=name, workspace=ws)
# logs = service.get_logs()

# for line in logs.split('\n'):
#     print(line)

# service.update(enable_app_insights=True)

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.


In [29]:
aci_service.delete()

# service.delete()

compute_target.delete()
